In [ ]:
import numpy as np
from datetime import datetime
from astropy.coordinates import EarthLocation, SkyCoord, AltAz
from astropy.time import Time
from astropy.io import fits
from astropy import units as u
from math import cos
from math import radians
from scipy.optimize import root_scalar
from math import pi

In [ ]:
# Раздел функций
def tograd(t):                                      # функция переводящая ra[h, m, s] в градусы
    return t[0]*15 + t[1]/60*15 + t[2]/3600*15



def RGB(x, y, z, am):                                   # функция расчета уравнения перехода от BVR к RGB с учетом экстицнии
    A = np.array([                                      # x = Bio, y = Gio, z = Rio
        [1.165, -0.165, 0],
        [0, 0.88, 0.12],
        [0, -0.378, 1.378]
    ])
    B = np.array([x-21.0612, y-20.7355, z-20.1309])     # x - Cb, y - Cv, z - Cr
    sol = np.linalg.solve(A, B)
    x, y, z = sol
    Bi = x + (0.2758 - 0.0037 * (x - y)) * am
    Gi = y + (0.2049 + 0.0054 * (y - z)) * am
    Ri = z + (0.1518 - 0.011 * (y - z)) * am
    return Bi, Gi, Ri

In [ ]:
# config file / данные из внешней оболочки
# magnitudes в B, V, R (Джонсоновские)
B, V, R = 12.26, 11.48, 11.48
latitude, longitude, obs_height = 43.6781, 41.4597, 2070        # координаты наблюдателя       
time_obs = [2024, 4, 8, 21, 12, 10]                             # Время наблюдений [год, месяц, день, час, минута, секунда]
ra_ha = [12, 42, 53]                                            # значения ra в часах, arcmin, arcsec
dec_deg = -0.495555 * u.deg
SN = 10                                                         # Signal / Noise
filter = "B"
# utcoffset = -3                                                # часовой пояс "-" for Eastern Daylight Time

In [157]:
# модуль расчета воздушной массы
ra_deg = tograd(ra_ha) * u.deg
# altitude = altitude * u.m                      # coordinates on Earth
# utcoffset = utcoffset * u.hour                                                       # часовой пояс

obs_location = EarthLocation.from_geodetic(lat=latitude, lon=longitude, height=obs_height)       # location of obervations

# observation time
obs_time = Time(datetime(time_obs[0], time_obs[1], time_obs[2], time_obs[3], time_obs[4], time_obs[5]),scale='utc',location=obs_location)   # возможна ошибка на 3часа

star_ICRS = SkyCoord(ra=ra_deg, dec=dec_deg)
star_altaz = star_ICRS.transform_to(AltAz(location=obs_location, obstime=obs_time))
z = 90 - star_altaz.alt.deg
AM = 1/cos(radians(z))


In [ ]:
# модуль записи коэффициентов модели


1.4054379081910684
(-8.488376602941074, -9.02884574868492, -8.611792836067426)
